# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/Users/gopinath/Documents/python_workspace/DataEngineering_NanoDegree/DataModeling/Project_2/DataModeling_Cassandra
['/Users/gopinath/Documents/python_workspace/DataEngineering_NanoDegree/DataModeling/Project_2/DataModeling_Cassandra/event_data/2018-11-15-events.csv', '/Users/gopinath/Documents/python_workspace/DataEngineering_NanoDegree/DataModeling/Project_2/DataModeling_Cassandra/event_data/2018-11-22-events.csv', '/Users/gopinath/Documents/python_workspace/DataEngineering_NanoDegree/DataModeling/Project_2/DataModeling_Cassandra/event_data/2018-11-09-events.csv', '/Users/gopinath/Documents/python_workspace/DataEngineering_NanoDegree/DataModeling/Project_2/DataModeling_Cassandra/event_data/2018-11-18-events.csv', '/Users/gopinath/Documents/python_workspace/DataEngineering_NanoDegree/DataModeling/Project_2/DataModeling_Cassandra/event_data/2018-11-04-events.csv', '/Users/gopinath/Documents/python_workspace/DataEngineering_NanoDegree/DataModeling/Project_2/DataModeling_Cassandra/event_

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print("Exception occured while connecting to database --->>>",e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify 
        WITH REPLICATION =
        { 'class' : 'SimpleStrategy', 'replication_factor': 1 }""")
except Exception as e:
    print("Exception occured while creating sparkify Keyspace --->>>>", e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print("Exception occured while setting keyspace --->>>>", e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Query 1 Data Modeling Design Decisions

   Query for Question 1 : 
       select artist_name, song_title, song_length from music_session_history where session_id = 338 and session_item_no = 4;
   <ol>
        <li> music_session_history table created with only selected fields to answer the question 1 </li>
        <li> PRIMARY KEY having session_id as partition key and session_item_id as the clustering key. </li>
   </ol>

In [8]:
# Query 1
query = """
    CREATE TABLE IF NOT EXISTS music_session_history
    (session_id int, session_item_id int,
     artist_name text, song_title text,
     song_duration float,
     PRIMARY KEY (session_id, session_item_id))"""
try:
    session.execute(query)
except Exception as e:
    print("Exception occured while creating music_session_history table --->>>", e)

In [9]:
file = 'event_datafile_new.csv'
query = """INSERT INTO music_session_history 
            (session_id, session_item_id, artist_name, 
             song_title, song_duration )
             VALUES (%s, %s, %s, %s, %s)"""

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader)
        for line in csvreader:
            session.execute(query, (int(line[8]), int(line[3]), line[0], 
                                    line[9], float(line[5])))
except Exception as e:
    print("Exception occured while inserting data into music_session_history --->>>>", e)

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = """SELECT artist_name, song_title, song_duration 
            from music_session_history where session_id = 338 and session_item_id= 2"""

try:
    rows = session.execute(query)
    print ("artist_name, song_title, song_duration")
    for row in rows:
        print (row.artist_name, ",", row.song_title, ",", row.song_duration)
except Exception as e:
    print("Query 1: Exception occured while retrieving data from music_session_history --->>>", e)


artist_name, song_title, song_duration
The Roots / Jack Davey , Atonement , 155.95057678222656


# Query 2 Data Modeling Design Decisions

   Query for Question 2 : 
           select artist_name, song_title, user_name from music_app_history where sessionid = 182, user_id =10;
    <ol>
        <li> music_app_history table created with all the fields from the data file to answer the question 2 </li>
        <li> PRIMARY KEY having session_id and user_id as partition key and session_item_id as the clustering key so that the data is sorted by session item id </li>
    </ol>

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Query 2
query = """
    CREATE TABLE IF NOT EXISTS music_user_session_history
    (user_id int, session_id int, session_item_id int,
     artist_name text, song_title text, user_name text,
     gender text, level text, song_duration float, location text,
     PRIMARY KEY ((user_id, session_id), session_item_id))"""
try:
    session.execute(query)
except Exception as e:
    print("Exception occured while creating music_user_session_history table --->>>", e)                 

In [12]:
file = 'event_datafile_new.csv'
query = """INSERT INTO music_user_session_history 
            (user_id, session_id, session_item_id, 
             artist_name, song_title, user_name, gender, 
             level, song_duration, location )
             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader)
        for line in csvreader:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], 
                                    line[9], line[1]+ " " + line[4], line[2], 
                                    line[6], float(line[5]), line[7]))
except Exception as e:
    print("Exception occured while inserting data into music_user_session_history --->>>>", e)

In [13]:
query = """SELECT artist_name, song_title, user_name  
            from music_user_session_history where  user_id =10 and session_id = 182"""

try:
    rows = session.execute(query)
    print ("artist_name, song_title, user_name")
    for row in rows:
        print (row.artist_name, ",", row.song_title, ",", row.user_name)
except Exception as e:
    print("Query 1: Exception occured while retrieving data from music_user_session_history --->>>", e)


artist_name, song_title, user_name
Down To The Bone , Keep On Keepin' On , Sylvie Cruz
Three Drives , Greece 2000 , Sylvie Cruz
Sebastien Tellier , Kilometer , Sylvie Cruz
Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Sylvie Cruz


# Query 3 Data Modeling Design Decisions

   Query for Question 3 : 
           select user_name from music_song_listeners where song_name='All Hands Against His Own'
    <ol>
        <li> music_song_listeners table created with only selected fields from the data file to answer the question 3 </li>
        <li> PRIMARY KEY having song_title as partition key and user_id as the clustering key </li>
    </ol>

In [14]:
# Query 3
query = """
    CREATE TABLE IF NOT EXISTS music_song_listeners
    (
     song_title text, user_id int, 
     song_duration float, user_name text,
     gender text, level text,  location text,
     PRIMARY KEY (song_title,user_id))"""
try:
    session.execute(query)
except Exception as e:
    print("Exception occured while creating music_song_listeners table --->>>", e)      

In [15]:
file = 'event_datafile_new.csv'
query = """INSERT INTO music_song_listeners 
            (song_title, user_id, song_duration, 
             user_name, gender, level, location)
             VALUES (%s, %s, %s, %s, %s, %s, %s)"""

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader)
        for line in csvreader:
            session.execute(query, (line[9], int(line[10]), float(line[5]), line[1]+ " " + line[4], line[2], 
                                    line[6], line[7]))
except Exception as e:
    print("Exception occured while inserting data into music_song_listeners --->>>>", e)

In [16]:
query = """SELECT user_name  
            from music_song_listeners where  song_title='All Hands Against His Own'"""

try:
    rows = session.execute(query)
    print ("user_name")
    for row in rows:
        print (row.user_name)
except Exception as e:
    print("Query 1: Exception occured while retrieving data from music_song_listeners --->>>", e)

user_name
Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
query = "DROP TABLE IF EXISTS music_session_history"

try:
    session.execute(query)
except Exception as e:
    print("Exception occured while dropping table music_session_history --->>>>", e)

In [18]:
query = "DROP TABLE IF EXISTS music_user_session_history"

try:
    session.execute(query)
except Exception as e:
    print("Exception occured while dropping table music_user_session_history --->>>>", e)

In [19]:
query = "DROP TABLE IF EXISTS music_song_listeners"

try:
    session.execute(query)
except Exception as e:
    print("Exception occured while dropping table music_song_listeners --->>>>", e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()